# Ajuster son premier modèle prédictif

Au cours de ce TD, vous serez amené·e à ajuster un modèle simple (et peu performant) pour prédire l’appartenance de tel ou tel manchot à une espèce pré-définie : manchot Adélie, manchot papou (*Gentoo*) ou manchot à jugulaire (*Chinstrap*). Vous mettrez pour cela en œuvre un algorithme d’apprentissage supervisé appelé *k-nearest neighbors* (k-NN) qui effectuera une comparaison entre les données envoyées pour la prédiction et les *k* observations les plus proches dans le jeu d’entraînement. 

La première tentative se révélera peu concluante, puis vous améliorerez progressivement la performance de votre algorithme en travaillant sur les données d’entraînement.

## À propos du jeu de données

![Meet the palmer penguins](../images/penguins.png)

Artwork by @allison_horst

Le fichier [*penguin-census.csv*](../data/penguin-census.csv) provient du paquet [*palmerpenguins*](https://allisonhorst.github.io/palmerpenguins/) conçu pour le logiciel *R* comme un outil pour l’exploration et la visualisation de données. Il contient deux jeux de données : un avec les données complètes et un autre qui se veut comme une version simplifiée du premier. C’est ce dernier que vous utiliserez.

Les données ont été collectées par le Dr. Kristen Gorman à la station Palmer en Antarctique.

> Gorman KB, Williams TD, Fraser WR (2014). Ecological sexual dimorphism and environmental variability within a community of Antarctic penguins (genus Pygoscelis). PLoS ONE 9(3):e90081. https://doi.org/10.1371/journal.pone.0090081

## Description des données

On n’insistera jamais assez sur l’influence d’une bonne compréhension des données à disposition sur la qualité de l’algorithme produit. Si vous avez l’impression de perdre du temps à explorer manuellement les fichiers ou à lire la documentation, rappelez-vous Hume : l’impression ne peut être représentative en ce qu’elle se présente immédiatement à l’esprit. L’idée qui en dérive, si elle a moins de force ou de vivacité, peut en revanche servir pour, en réaction, modifier l’impression primitive. En somme, non, vous ne perdez pas de temps.

Les données sont constituées de 344 observations (*samples*) décrites par 8 variables aléatoires, ou caractéristiques (*features*). Ces variables sont :

|Variable|Signification|Type de variable|
|-|-|:-:|
|*species*|Espèce de manchot parmi : Adelie, Gentoo, Chinstrap|qualitative|
|*island*|Île de l’observation parmi : Torgersen, Biscoe, Dream|qualitative|
|*bill_length_mm*|Longueur du bec de l’individu (en mm)|quantitative discrète|
|*bill_depth_mm*|Épaisseur du bec de l’individu (en mm)|quantitative discrète|
|*flipper_length_mm*|Longueur de la nageoire de l’individu (en mm)|quantitative discrète|
|*body_mass_g*|Poids de l’individu (en g)|quantitative discrète|
|*sex*|Sexe de l’individu selon deux valeurs possibles : *male* ou *female*|qualitative binaire|
|*year*|Année de l’observation (de 2007 à 2009)|qualitative ordonnée|

### Définition de la tâche

L’objectif de votre programme est de concevoir une fonction qui détermine si un individu appartient à telle ou telle espèce de manchot. Comme vous ne savez pas encore comment traiter des données catégorielles (sexe, profession, statut, pour ou contre…), vous limiterez les données d’entraînement aux variables quantitatives.

**Variable cible (*target*) :** *species*  
**Caractéristiques (*features*) :** *bill_length_mm*, *bill_depth_mm*, *flipper_length_mm*, *body_mass_g*

## Un coup d’œil rapide aux données

### Chargement du fichier

Importez tout d’abord le fichier *penguin-census.csv*, situé dans le répertoire *data*,  grâce à la méthode `.read_csv()` de la librairie *Pandas* :

In [ ]:
# your code here

# load pandas library
import pandas as pd

# load the census
penguin_census = pd.read_csv("../data/penguin-census.csv")

### Exploration des données

Avant toute chose, confirmez le nombre d’individus qui constituent le jeu de données :

In [ ]:
# your code here

# number of samples
len(penguin_census)

La méthode `.head()` permet d’obtenir un aperçu des premières observations. Utilisez-là sur la variable dans laquelle vous avez chargé le *data frame* :

In [ ]:
# your code here

# look at the data
penguin_census.head()

Une propriété intéressante des *data frames* est `dtypes`, qui fournit le type de donnée de chaque variable :

In [ ]:
# your code here

# data types
penguin_census.dtypes

Utilisez à présent la méthode `.describe()` pour obtenir des mesures statistiques sur les variables quantitatives :

In [ ]:
# your code here

# some statistic measures on quantitative features
penguin_census.describe()

### À la recherche des données manquantes

Plus haut, dès l’utilisation de la méthode `.head()`, vous avez révélé l’absence de certaines données. L’information est confirmée par la méthode `.describe()` qui ne totalise que 342 observations valides pour les variables *bill_length_mm*, *bill_depth_mm*, *flipper_length_mm* et *body_mass_g* sur les 344 individus.

Une façon plus sûre de les détecter est d’utiliser la méthode `.isnull()` :

In [ ]:
# your code here

# which values are null in the dataset?
penguin_census.isnull()

Mieux encore, chaînez l’opération précédente avec la méthode `.any()` pour déterminer dans quelles colonnes on trouve des données manquantes :

In [ ]:
# your code here

# in which columns are the missing values?
penguin_census.isnull().any()

Vous pouvez maintenant pister, colonne par colonne, les données manquantes. La syntaxe se construit de la manière suivante :
1. Appliquez un prédicat à votre *data frame*  
`df[]` ;
2. dans le prédicat, limitez la sélection à la colonne appropriée  
`df[df[col]]` ;
3. révélez les valeurs manquantes de la colonne sélectionnée avec `.isnull()`  
`df[df[col].isnull()]` ;
4. ne retenez que les indices des observations sélectionnées (en dehors du prédicat) avec la propriété `index`  
`df[df[col].isnull()].index` ;
5. transformez enfin le tout en liste avec la méthode `.to_list()`  
`df[df[col].isnull()].index.to_list()`.

In [ ]:
# your code here

# where are precisely the missing values?
[
    f"{col} : {penguin_census[penguin_census[col].isnull()].index.to_list()}"
    for col in ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']
]

### Visualiser les données

La librairie *Pandas* embarque des outils hérités de *Matplotlib*. La méthode `.hist()` est toute indiquée pour représenter des variables numériques. Notez qu’il n’est pas nécessaire d’opérer colonne par colonne. *Pandas* n’applique la méthode que sur des variables de type numérique : 

In [ ]:
# your code here

_ = penguin_census.hist(figsize=(8,8))

Mieux encore, la librairie *Seaborn* offre des outils qui facilitent davantage la lecture des graphiques. Importez tout d’abord le module `seaborn` avec l’alias `sn`, puis utilisez la méthode `.pairplot()` en lui passant deux paramètres : votre *data frame* et le nom de la variable cible (paramètre nommé `hue`) :

In [ ]:
# your code here

# module seaborn
import seaborn as sn

# combination of plots
_ = sn.pairplot(penguin_census, hue="species")

## Préparez le jeu de données

Après avoir consacré un temps non négligeable à la compréhension du jeu de données, il est temps de le préparer afin de fournir un *data frame* propre à votre futur programme.

Commencez par définir la colonne cible dans une variable *target* et par faire la liste des caractéristiques qui vous intéressent dans une variable *features* :

In [ ]:
# your code here

target = "species"
features = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]

Le nouveau *data frame* correspondra à la réunion des seules colonnes répertoriées dans vos variables *target* et *features*. Par convention, on positionne la colonne cible à la fin :

In [ ]:
# your code here

# a new data frame
data = penguin_census[features + [target]]

Lorsque vous avez fait la chasse aux données manquantes dans le fichier, vous avez sans doute remarqué que :
- les colonnes qui vous intéressent ne contenaient que deux données manquantes ;
- ces données concernent les deux mêmes individus.

Vous pouvez de manière légitime considérer que supprimer totalement ces deux individus ne nuira pas à votre programme. Appliquez la méthode `.dropna()` directement sur le *data frame* :

In [ ]:
# your code here

data = data.dropna()

Votre *data frame* n’est plus constitué que de 342 observations. Il est temps de le séparer en deux autres structures :
1. une série nommée `y` pour la cible ;
2. un *data frame* nommé `X` pour les caractéristiques (sans la colonne cible).

La méthode `.drop()` avec l'argument nommé `columns` permet de spécifier une liste de colonnes à supprimer.

In [ ]:
# your code here

y = data[target]
X = data.drop(columns=[target])

## Constituez les jeux d’entraînement et de test

Durant cette étape, vous allez séparer vos structures `X` et `y` en deux parties : la première pour l’entraînement, la seconde pour le test.

Comptabilisez dans une variable `nb` le nombre d’observations et définissez une variable `limit` qui soit 25 % de `nb`. Assurez-vous que `limit` soit un entier :

In [ ]:
# your code here

nb = len(X)
limit = int(nb * 0.25)

Grâce au *slicing*, effectuez la séparation entre `X_train`, `X_test` et `y_train`, `y_test` :

In [ ]:
# your code here

# split train/test
X_train = X[limit:]
X_test = X[:limit]
y_train = y[limit:]
y_test = y[:limit]

## Programmation du modèle

### Entraîner le modèle avec le jeu d’entraînement

La partie qui aurait pu sembler la plus complexe est en fait la plus triviale. *Scikit-Learn* se charge de calculer l’ensemble des paramètres et fournit un modèle selon l’algorithme d’entraînement utilisé.

Importez tout d’abord la classe `KNeighborsClassifier` du module `sklearn.neighbors` :

In [ ]:
# your code here

from sklearn.neighbors import KNeighborsClassifier

Chargez à présent la classe `KNeighborsClassifier()` dans une variable `model` puis appelez directement la méthode `.fit()` en lui passant en paramètres `X_train` puis `y_train` :

In [ ]:
# your code here

model = KNeighborsClassifier()
model.fit(X_train, y_train)

### Tester le modèle

Il ne reste plus qu’à effectuer des prédictions sur les données de test avec la méthode `.predict()` :

In [ ]:
# your code here

y_predicted = model.predict(X_test)

### Évaluer le modèle

Pour évaluer l’exactitude du modèle, il suffit de comparer les prédictions obtenues sur le jeu de test avec le contenu de `y_test`, puis de noter les concordances. L’opérateur d’égalité `==` permet de vérifier si les observations d’une série correspondent avec celles d’une autre : 

In [ ]:
# your code here

y_predicted == y_test

Le résultat obtenu étant une série de booléens, il est possible d’appeler les méthodes `.sum()` ou `.mean()` pour obtenir le nombre de bonnes prédictions rapportées au nombre total d’éléments :

In [ ]:
# your code here

(y_predicted == y_test).mean()

Mieux, la méthode `.score()` du modèle entraîné permet de révéler le résultat immédiatement. Passez simplement en paramètres le jeu de test et la cible de test :

In [ ]:
# your code here

model.score(X_test, y_test)

### Améliorer le modèle

**49 % de taux d’exactitude !!** Même pas une prédiction sur deux de juste, autant dire que votre premier modèle est ridiculement inefficace…

La première voie d’amélioration réside dans la préparation du jeu de données. Remarquez que vos données sont triées par espèces de manchots. Si vous ressortez les données uniques de `y_test`, vous remarquez alors qu’il n’y a que des manchots Adélie :

In [ ]:
# unique values in y_test
y_test.unique()

Alors que dans la série cible d’entraînement les manchots papous sont sur-représentés :

In [ ]:
# nb of samples for each class in y_train
y_train.value_counts()

La librairie *Pandas* met à disposition une méthode `.sample()` qui permet de mélanger les observations :

In [ ]:
# shuffle all rows
data = data.sample(frac=1).reset_index(drop=True)

Séparez à nouveau votre *data frame* en `X` et `y`, puis en jeux de test et d’entraînement, et entraînez le modèle tel quel. Vous devriez noter une hausse non négigeable du taux d’exactitude !

Notez qu’il existe une fonction dans *Scikit-Learn* pour préparer automatiquement les données de test et d’entraînement :

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)

N’hésitez pas à tester d’autres pistes d’amélioration, vous ne devriez pas tarder à dépasser les 80 % d’exactitude :
- Modifier le paramètre `random_state` de la fonction `train_test_split()` ;
- augmenter le nombre d’observations dans le jeu d’entraînement (ex. : 80/20) ;
- ajuster le paramètre `n_neighbors` de la classe `KNeighborsClassifier()` (5 par défaut) ;
- …